<a href="https://colab.research.google.com/github/cu-mkp/word-embeddings/blob/main/WordEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting drive

! git clone https://github.com/cu-mkp/manuscript-object
! git clone https://github.com/cu-mkp/m-k-manuscript-data


fatal: destination path 'manuscript-object' already exists and is not an empty directory.
fatal: destination path 'm-k-manuscript-data' already exists and is not an empty directory.


In [2]:
%cd /Users/sophiaqureshi/manuscript-object/

/Users/sophiaqureshi/manuscript-object


In [ ]:
! python --version

Python 3.8.5


In [3]:
! pipenv install --python 3.8.5

Virtualenv already exists!
Removing existing virtualenv...
Creating a virtualenv for this project...
Pipfile: /Users/sophiaqureshi/manuscript-object/Pipfile
Using /Users/sophiaqureshi/opt/anaconda3/bin/python3.8 (3.8.5) to create virtualenv...
⠙ Creating virtual environment...created virtual environment CPython3.8.5.final.0-64 in 645ms
  creator CPython3Posix(dest=/Users/sophiaqureshi/.local/share/virtualenvs/manuscript-object-HgciJDnK, clear=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/Users/sophiaqureshi/Library/Application Support/virtualenv)
    added seed packages: pip==20.2.3, setuptools==50.3.0, wheel==0.35.1
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator

✔ Successfully created virtual environment! 
Virtualenv location: /Users/sophiaqureshi/.local/share/virtualenvs/manuscript-object-HgciJDnK
Installing dependencies from Pipfile.lock (31d

In [4]:
from manuscript import *
import utils

m = Manuscript.from_dirs(*utils.version_paths)

properties = ['animal', 'body_part', 'currency', 'definition', 'environment', 'material',
              'medical', 'measurement', 'music', 'plant', 'place', 'personal_name',
              'profession', 'sensory', 'tool', 'time']
  


Streaming output truncated to the last 5000 lines.
Generating entry with folio 066r, ID p066r_5...
Generating entry with folio 096v, ID p096v_1...
Generating entry with folio 096v, ID p096v_2...
Generating entry with folio 096v, ID p096v_3...
Generating entry with folio 109v, ID p109r_5...
Generating entry with folio 109v, ID p109v_1...
Generating entry with folio 109v, ID p109v_2...
Generating entry with folio 118r, ID p118r_1...
Generating entry with folio 118r, ID p118r_2...
Generating entry with folio 118r, ID p118r_3...
Generating entry with folio 118r, ID p118r_4...
Generating entry with folio 118r, ID p118r_5...
Generating entry with folio 087r, ID p087r_1...
Generating entry with folio 168r, ID p167r_1...
Generating entry with folio 136r, ID p135v_1...
Generating entry with folio 136r, ID p136r_1...
Generating entry with folio 045v, ID p045v_1...
Generating entry with folio 045v, ID p045v_2...
Generating entry with folio 127v, ID p127v_1...
Generating entry with folio 054r, ID 

In [5]:
words_tl = {}
words_tcn = {}

#loops through each entry in the manuscript and extracts the values for each property
#for each word in the property list, adds an entry to the dictionary by language
#each word is mapped to a dictionary where each key is an entry id and each value is a list of semantic tags 
#associated with that word in the entry
for lang, entries in m.entries.items():
  if lang == "tl":
    for entry in entries.values():
      for prop in properties:
        pwords = entry.properties[prop] 
        for pw in pwords: 
          if pw not in words_tl:
            words_tl[pw] = {}
          if entry.identity not in words_tl[pw]:
            words_tl[pw][entry.identity]=[]
          words_tl[pw][entry.identity].append(prop)
  if lang == "tcn":
    for entry in entries.values():
      for prop in properties:
        pwords = entry.properties[prop] 
        for pw in pwords: 
          if pw not in words_tcn:
            words_tcn[pw] = {}
          if entry.identity not in words_tcn[pw]:
            words_tcn[pw][entry.identity]=[]
          words_tcn[pw][entry.identity].append(prop)

print(words_tl)
print(words_tcn)


    
  

{'swan': {'24r1': ['animal']}, 'face': {'24r1': ['body_part'], '20v1': ['body_part'], '123r1': ['body_part', 'body_part'], '108r2': ['body_part'], '5r2': ['body_part'], '77r1': ['body_part'], '33r1': ['body_part'], '79v2': ['body_part'], '103r1': ['body_part']}, 'wounds': {'24r1': ['body_part', 'medical'], '21v2': ['medical']}, 'finger': {'24r1': ['body_part'], '86v1': ['body_part', 'tool'], '66r6': ['body_part'], '3r2': ['body_part', 'tool'], '31r4': ['body_part', 'tool'], '20v6': ['body_part', 'measurement'], '137v1': ['body_part', 'tool'], '34r1': ['body_part', 'body_part'], '107r1': ['body_part', 'measurement'], '72v1': ['body_part', 'tool'], '52r1': ['body_part', 'measurement'], '139v1': ['body_part', 'tool'], '94r1': ['body_part', 'body_part', 'tool'], '144r1': ['body_part', 'tool'], '37r2': ['body_part', 'measurement'], '139r1': ['body_part', 'tool'], '5r2': ['body_part', 'body_part'], '106v2': ['body_part', 'measurement'], '23r1': ['body_part', 'body_part', 'measurement', 'meas

In [ ]:
for word in words_tl:
  print(word + ":")
  

In [ ]:
#install Huggingface transformers

!pip3 install transformers

In [35]:
#from https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
import torch
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# tokenizing an example entry
e = m.entries['tl']['24r1'].text

#preprocess the text 
e_lowercase = e.lower()
e_split = e_lowercase.split()
#print(e_split)

In [ ]:
e_split.pop(0) #removing title of entry
e_sentences = []
current_sentence = []
for w in e_split:
  current_sentence.append(w)
  if "." in w:
    current_sentence = " ".join(current_sentence)
    current_sentence = "[CLS] " + current_sentence + " [SEP]"
    e_sentences.append(current_sentence)
    current_sentence = []

print(e_sentences)

['[CLS] the finest metal one can find, for there is fine metal and crude metal. [SEP]', '[CLS] fine metal is that of a large bell, because one puts in more copper to give it a bigger voice, & for small bells one puts more tin to give it a <-bigger-> clearer sound. [SEP]', '[CLS] the metal of a large bell is made <-one-> with 3 quintals of rosette & twenty or xxv lb of tin. [SEP]', '[CLS] <-tan-> each grenade must weigh four to six lb. [SEP]', '[CLS] and in order to throw them, it is necessary that they be full of powder mixed with coarsely pestled glass. [SEP]', '[CLS] and for a quarter lb of powder, one needs half an ounce of glass which is put with it, in order that on the face or places where the powder will have its effect, the wounds will be all the worse for it. [SEP]', '[CLS] the best arquebus powder is required here. [SEP]', '[CLS] the hole needs to be as large as the thickness of a swan’s quill, & you need not bother to make it threaded, but it is enough to make it even. [SEP]

In [ ]:
marked_text = e_sentences[0]

tokenized_text = tokenizer.tokenize(marked_text)
print(tokenized_text)

['[CLS]', 'the', 'finest', 'metal', 'one', 'can', 'find', ',', 'for', 'there', 'is', 'fine', 'metal', 'and', 'crude', 'metal', '.', '[SEP]']


In [ ]:
tokenized_batch = tokenizer(e_sentences, padding=True, return_tensors="pt")
print(tokenized_batch)

In [ ]:
#load the model for English
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

with torch.no_grad():
  outputs = model(**tokenized_batch)

  # Evaluating the model will return a different number of objects based on 
  # how it's  configured in the `from_pretrained` call earlier. In this case, 
  # becase we set `output_hidden_states = True`, the third item will be the 
  # hidden states from all layers. See the documentation for more details:
  # https://huggingface.co/transformers/model_doc/bert.html#bertmodel

  hidden_states = outputs[2]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(output)

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 17
Number of tokens: 52
Number of hidden units: 768


In [ ]:
#extract sentences for each tagged word (tl)

entries_by_word = {}

for word in words_tl:
  tokenized_entries = []
  for entry in words_tl[word]:

    ce = m.entries['tl'][entry].text
    tokenized_entry = tokenizer.tokenize(ce)
    tokenized_entries.append(tokenized_entry)
  entries_by_word[word] = tokenized_entries
